# Mnist CGAN

In [ ]:
import os
import sys

import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F

import torchvision 
import torchvision.transforms as tv_transforms
import torchvision.datasets as tv_datasets
import torchvision.utils as tv_utils

from torch.utils.tensorboard import SummaryWriter

from fid import calculate_activation_statistics
from models import MnistCNN, Discriminator, Generator
from inception import InceptionV3
from datasets import ColorMNIST
from plot_tools import plot_im
from utils import makedirs_exists_ok, seed_rng, set_cuda_visible_devices, load_weights_from_file

In [ ]:
model_name = 'projection_cgan'
data_root = './data'
model_root = './models/mnist_cgan'
figure_root = './figures/mnist_cgan'
log_root = './logs/mnist_cgan'
image_size = 32
batch_size = 32
n_workers = 1
seed = 9
gpu_id = '0'
n_workers = 4
load_weights = ''
lr = 0.0002
beta1 = 0
beta2 = 0.9
n_epochs = 20
log_interval = 10
target_type = 'color'


n_features = 64
G_dim_z = 64
G_bottom_width = 4
n_classes = 10


In [ ]:
makedirs_exists_ok(data_root)
makedirs_exists_ok(model_root)
makedirs_exists_ok(figure_root)
makedirs_exists_ok(log_root)

writer = SummaryWriter(log_root)
writer.flush()

seed_rng(seed)
device = set_cuda_visible_devices(gpu_id)

In [ ]:
transforms = tv_transforms.Compose([
    tv_transforms.Resize(image_size),
    tv_transforms.ToTensor(),
    tv_transforms.Normalize((0.1307,), (0.3081,)),
])

train_loader = torch.utils.data.DataLoader(
    ColorMNIST(
        root=data_root, download=True, train=True, transform=transforms),
    batch_size=batch_size, shuffle=True, num_workers=n_workers, pin_memory=True)
test_loader = torch.utils.data.DataLoader(
    ColorMNIST(
        root=data_root, download=True, train=False, transform=transforms),
    batch_size=batch_size, shuffle=True, num_workers=n_workers, pin_memory=True)

In [ ]:
G = Generator(n_features, G_dim_z, G_bottom_width, num_classes=n_classes).to(device)
D = Discriminator(n_features, num_classes=n_classes).to(device)
print(G,D)

In [ ]:

optimizer_G = torch.optim.Adam(G.parameters(), lr, (beta1, beta2))
optimizer_D = torch.optim.Adam(D.parameters(), 0.00005, (beta1, beta2))

# hinge loss
criterion_G = lambda D_xf, D_xr: -torch.mean(D_xf)
criterion_D = lambda D_xf, D_xr: \
    torch.mean(torch.relu(1. - D_xf)) + \
    torch.mean(torch.relu(1. + D_xr))

criterion_G = lambda D_xf, D_xr: torch.mean(F.softplus(-D_xf))
criterion_D = lambda D_xf, D_xr: torch.mean(F.softplus(-D_xr)) + torch.mean(F.softplus(D_xf))


def sample_z(batch_size, dim_z, device, z_distribution='normal'):    
    z = torch.empty(batch_size, dim_z, dtype=torch.float32, device=device)
    if z_distribution == 'normal':
        return z.normal_()
    else:
        return z.uniform_()
    
def sample_y(batch_size, n_classes, device):
    """ conditioned variable """
    y = torch.from_numpy(
        np.random.randint(low=0, high=n_classes, size=(batch_size,)))
    y = y.type(torch.long).to(device)
    return y

def sample_from_G(G, batch_size, dim_z, device, n_classes, z_distribution='normal'):
    z = sample_z(batch_size, dim_z, device, z_distribution)
    y = sample_y(batch_size, n_classes, device)
    x_fake = G(z, y)
    return x_fake, y, z
    

In [ ]:
fixed_z = sample_z(64, G_dim_z, device)
fixed_y = sample_y(64, n_classes, device)

for epoch in range(n_epochs):
    for it, (x, y, c) in enumerate(train_loader):
        
#         # color
#         x, y = x.to(device), (c < 0.5).long().to(device)
        # style 
        x, y = x.to(device), y.long().to(device)
        
        # Generator
        
        x_fake, y_rand, _ = sample_from_G(G, batch_size, G_dim_z, device, n_classes)
        D_xf = D(x_fake, y_rand)
        loss_G = criterion_G(D_xf, None)
        
        G.zero_grad()
        loss_G.backward()
        optimizer_G.step()
        
        # Discriminator
        
        x_fake, y_rand, _ = sample_from_G(G, batch_size, G_dim_z, device, n_classes)
        D_xf = D(x_fake, y_rand)
        D_xr = D(x, y)
        loss_D = criterion_D(D_xf, D_xr)
        
        D.zero_grad()
        loss_D.backward()
        optimizer_D.step()
        

        ##############################################################
        # print
        ##############################################################


        loss_D = loss_D.item()
        loss_G = loss_G.item()
        loss_total = loss_D + loss_G

        global_step = epoch*len(train_loader)+it
        writer.add_scalar('loss/total', loss_total, global_step)
        writer.add_scalar('loss/D', loss_D, global_step)
        writer.add_scalar('loss/G', loss_G, global_step)

        if it % log_interval == log_interval-1:
            print(f'[{epoch+1}/{n_epochs}]\t'
                  f'[{(it+1)*batch_size}/{len(train_loader.dataset)} ({100.*(it+1)/len(train_loader):.0f}%)]\t'
                  f'loss: {loss_total:.4}\t'
                  f'loss_D: {loss_D:.4}\t'
                  f'loss_G: {loss_G:.4}\t')
            
            x_fake = G(fixed_z, fixed_y).detach()
            tv_utils.save_image(x_fake,
                os.path.join(figure_root,
                    f'{model_name}_fake_samples_epoch={epoch}_it={it}.png'))

            writer.add_image('mnist', tv_utils.make_grid(x_fake), global_step)
        

#     torch.save(G.state_dict(), os.path.join(model_root, f'G_epoch_{epoch}.pt'))
#     torch.save(D.state_dict(), os.path.join(model_root, f'D_epoch_{epoch}.pt'))


In [ ]:
torch.min(x_fake), torch.max(x_fake)
plot_im(x_fake)